This notebook collects key steps taken to produce the training dataset that:
* is aggregated to 1x1sqkm (from by block data)
* has POI features as surfaces
* has features for urban area characteristics

## dependencies

In [1]:
import pathlib
import sys
sys.path.insert(0, '../utils')

import geoutils
import bqutils
from settings import *

## workspace

In [2]:
dirs = [feats_dir, inds_dir]
for dir_ in dirs:
    with pathlib.Path(dir_) as path:
        if not path.exists():
            path.mkdir(parents=True, exist_ok=True)

## data

In [3]:
# !gsutil cp gs://immap-masks/admin_boundaries/admin_bounds.gpkg {feats_dir}
# !gsutil cp gs://immap-wash-training/features/2018_{area}_*.tif {feats_dir}
# !gsutil cp gs://immap-wash-training/features/urban_area_features.csv {feats_dir}
# !gsutil cp gs://immap-wash-training/indicators/Manzanas_urbano.zip {feats_dir}
# !gsutil cp gs://immap-wash-training/indicators/indicator_labelled_grid.csv {inds_dir}
# !gsutil cp gs://immap-wash-training/grid/grid_1x1km_wfeatures_lagged.csv {feats_dir}
# !unzip {feats_dir}Manzanas_urbano.zip

## indicators

In [4]:
gdf = geoutils.generate_indicator_labelled_grid()

## features

In [5]:
# # poi features - done in BQ
# depts = get_depts()
# for poi in pois:
#     print(f'Processing {poi}')
#     geoutils.generate_poi_features_by_dept(poi)

In [6]:
gdf = geoutils.generate_satellite_features(gdf)

100%|██████████| 13/13 [00:24<00:00,  1.92s/it]


In [7]:
# # Urban area + Spatial lag features - done in BQ or separate notebook
# geoutils.generate_urban_area_features()
# geoutils.generate_spatial_lag_features()

In [8]:
train_df = geoutils.generate_training_data(gdf)
print(train_df.shape)
train_df.head(2)

(7574, 65)


,pixelated_urban_area_id,id,geometry,perc_hh_no_toilet,perc_hh_no_water_supply,perc_hh_no_sewage,d_mc_basur,d_mc_aguac,d_mc_freq_,d_mc_pare,...,lag_aridity_cgiarv2,lag_temperature,lag_nighttime_lights,lag_population,lag_elevation,lag_urban_index,lag_nearest_highway,nighttime_lights_area_mean,x,y
0,862.0,417475,"POLYGON((-75.5123828117681 5.05751500688412, -...",0.018677,0.020431,0.030647,0.029925,0.150449,0.793726,0.221855,...,17281.00001,14980.00001,47.137354,75.599635,2032.50001,31.37501,381.252504,23.124894,-75.507891,5.062007
1,83.0,187318,"POLYGON((-76.4376475501431 7.23143798441016, -...",0.190164,0.213115,0.209836,0.062295,0.409836,0.760656,0.501639,...,21134.00001,15023.68751,0.483760,1.064785,175.37501,6.87501,727.636400,0.808125,-76.433156,7.235930


In [38]:
train_df.to_csv(data_dir + '20200821_dataset.csv', index = False)
!gsutil cp {data_dir}20200821_dataset.csv gs://immap-wash-training/training/

Copying file://../data/20200821_dataset.csv [Content-Type=text/csv]...
/ [1 files][  7.6 MiB/  7.6 MiB]                                                
Operation completed over 1 objects/7.6 MiB.                                      
